## code that runs inference daily and gets all the feats it needs

finally it stores them in predictions hopsworks

In [1]:
from Features.df_functions import *
from Features.inference import *
from Models.Bird_percent import *
from hsfs.feature import Feature
import datetime
import requests
import pandas as pd
import hopsworks
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
load_dotenv()

warnings.filterwarnings("ignore")
from dotenv import load_dotenv
load_dotenv()

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login(project="BirdUp", api_key_value=HOPSWORKS_API_KEY)

2025-12-21 14:28:33,718 INFO: Initializing external client
2025-12-21 14:28:33,719 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-12-21 14:28:35,888 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1327256
2025-12-21 14:28:36,795 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-12-21 14:28:36,797 INFO: Initializing external client
2025-12-21 14:28:36,797 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-12-21 14:28:38,198 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1327256


In [2]:
## Feature Transforms
# get the df locally from hopsworks

import pandas as pd
import hopsworks
import os
import warnings
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from tqdm import tqdm_notebook as tqdm
from dataclasses import dataclass



@dataclass
class DataSet:
    train: pd.DataFrame
    val: pd.DataFrame
    test: pd.DataFrame

    @property
    def train_tensor(self) -> torch.Tensor:
        return torch.tensor(self.train.to_numpy(), dtype=torch.float32)

    @property
    def val_tensor(self) -> torch.Tensor:
        return torch.tensor(self.val.to_numpy(), dtype=torch.float32)

    @property
    def test_tensor(self) -> torch.Tensor:
        return torch.tensor(self.test.to_numpy(), dtype=torch.float32)


def get_feature_group_as_df(feature_group_name: str = "birding", feature_group_version: int = 1):
    fs = project.get_feature_store()
    fg = fs.get_feature_group(
        name=feature_group_name, version=feature_group_version
    )
    df = fg.read()
    return df

def sort_by_date(df:pd.DataFrame, date_col:str='observation_date'):
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(by=date_col).reset_index(drop=True)
    return df

def true_false_to_01(df:pd.DataFrame):
    # Vectorized conversion is significantly faster than lambda mapping
    return df.astype({col: int for col in df.select_dtypes(include='bool').columns})

def encode_reigon(df:pd.DataFrame, region_col:str='region', premade_mapping:dict=None):
    # calculate means based on observation_count
    if premade_mapping is not None:
        region_mapping = premade_mapping
        df[region_col] = df[region_col].map(region_mapping)
        return df, region_mapping

    region_means = df.groupby(region_col)['observation_count'].mean().sort_values()
    
    # map to ordinal integers based on rank (0 = lowest avg count, 24 = highest)
    region_mapping = {region: i for i, region in enumerate(region_means.index)}
    df[region_col] = df[region_col].map(region_mapping)
    
    return df, region_mapping

def get_reigon_mapping():
    map ={'Dalsland': 0, 'Ångermanland': 1, 'Medelpad': 2, 'Jämtland': 3, 'Härjedalen': 4, 'Bohuslän': 5, 'Hälsingland': 6, 'Dalarna': 7, 'Norrbotten': 8, 'Blekinge': 9, 'Lappland': 10, 'Värmland': 11, 'Västmanland': 12, 'Gästrikland': 13, 'Småland': 14, 'Östergötland': 15, 'Närke': 16, 'Västerbotten': 17, 'Gotland': 18, 'Västergötland': 19, 'Halland': 20, 'Uppland': 21, 'Öland': 22, 'Södermanland': 23, 'Skåne': 24}
    return map

def drop_unused_columns(df:pd.DataFrame, drop_date:bool=True):
    unused_cols = ["observation_date", "time_observations_started", "weathercode", "obs_count_lag_1", "obs_count_lag_2", "obs_count_lag_3", "obs_count_lag_4", "obs_count_lag_5"]
    if not drop_date:
        unused_cols.remove("observation_date")
    df = df.drop(columns=unused_cols)
    return df

def birdcount_binarization(df:pd.DataFrame):
    df['bird_count_binary'] = (df['observation_count'] > 0).astype(int)
    df = df.drop(columns=['observation_count'])
    return df

def split_data(df:pd.DataFrame, train_size:float=0.8, val_size:float=0.1, shuffle = True) -> dict[str, DataSet]:
    # ------Helper function -----
    def show_dataset_end_dates(data_dict: dict):
        for bird, ds in data_dict.items():
            print(f"Bird: {bird}")
            # Check train, val, and test splits
            for split_name in ['train', 'val', 'test']:
                df = getattr(ds, split_name)
                
                if 'observation_date' in df.columns:
                    end_val = int(df['observation_date'].max()) 
                elif 'year' in df.columns:
                    # Fallback if date was dropped but year remains
                    end_val = f"Year: {df['year'].max()}"
                else:
                    end_val = "Date column not found"
                    
                print(f"  {split_name.capitalize()} End: {end_val}")

    # ---- Main function body ----            
    if shuffle:
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    unique_birds=df["bird_type"].unique()
    # make dict with the keys train, val, test and empty dataframes as values
    ret_dict = {}
    for bird in unique_birds:
        bird_df = df[df["bird_type"]==bird]
        bird_df = bird_df.drop(columns=["bird_type"]).reset_index(drop=True)
        n = bird_df.shape[0]
        train_end = int(n * train_size)
        val_end = int(n * (train_size + val_size))
        
        bird_df_train = bird_df.iloc[:train_end]
        bird_df_val = bird_df.iloc[train_end:val_end]
        bird_df_test = bird_df.iloc[val_end:]
        
        ret_dict[bird] = DataSet(train=bird_df_train, val=bird_df_val, test=bird_df_test)
    if not shuffle:
        print("Dataset end dates (no shuffle):")
        show_dataset_end_dates(ret_dict)
    return ret_dict

def drop_x_negative_samples(df, drop_percentage=0.2):
    """
    Here we downsample the negative samples (bird_count_binary == 0) by a given percentage.
    This helps to balance the dataset, we have a large number of negative samples compared to positive ones.
    """
    neg = df[(df['bird_count_binary'] == 0)]
    pos = df[~((df['bird_count_binary'] == 0))]
    
    # Calculate number of samples to drop
    num_to_drop = int(len(neg) * drop_percentage)
    
    # Randomly sample the indices to keep
    neg_downsampled = neg.sample(n=len(neg) - num_to_drop, random_state=42)
    
    # Combine back and shuffle
    new_df = pd.concat([pos, neg_downsampled]).sample(frac=1, random_state=42).reset_index(drop=True)
    
    return new_df


def make_feature_compatible(df:pd.DataFrame, reference_df:pd.DataFrame):
    # ensure the dataframe has all the feats and as many rows in the right order
    cols_to_add =['sighted_lag_5', 'sighted_lag_3', 'sighted_lag_4', 'sighted_lag_2', 'bird_count_binary', 'sighted_lag_1']
    for col in cols_to_add:
        if col not in df.columns:
            df[col] = 0
    # dupplicate all rows and have half of them bird_type = "whteag" and half = goleag
    df = pd.concat([df.assign(bird_type='whteag'), df.assign(bird_type='goleag')], ignore_index=True)
    # sort the colums like the reference df
    df = df[reference_df.columns]
    return df

def fill_sighted_lag(filled: pd.DataFrame, original: pd.DataFrame):
    filled['observation_date'] = pd.to_datetime(filled['observation_date'])
    original['observation_date'] = pd.to_datetime(original['observation_date'])
    
    # Combine datasets to ensure rolling windows carry over from history to future
    combined = pd.concat([original, filled]).drop_duplicates(['observation_date', 'bird_type', 'region'], keep='last')
    combined = combined.sort_values(['bird_type', 'region', 'observation_date'])

    # Shift bird_count_binary within each group to create lags
    for i in range(1, 6):
        combined[f'sighted_lag_{i}'] = combined.groupby(['bird_type', 'region'])['bird_count_binary'].shift(i)

    # Update filled dataframe with calculated lags
    filled = filled.drop(columns=[f'sighted_lag_{i}' for i in range(1, 6)])
    filled = filled.merge(
        combined[['observation_date', 'bird_type', 'region'] + [f'sighted_lag_{i}' for i in range(1, 6)]],
        on=['observation_date', 'bird_type', 'region'],
        how='left'
    )
    # convert sighted_lag columns to int
    for i in range(1, 6):
        filled[f'sighted_lag_{i}'] = filled[f'sighted_lag_{i}'].astype(int)
    return filled

def format_observation_date(df: pd.DataFrame):
    df['observation_date'] = pd.to_datetime(df['observation_date']).dt.strftime('%Y-%m-%d') # Changed: Formatted date to y-m-d string
    return df

def divide_ds(ds:pd.DataFrame):
    goleag_ds = ds[ds["bird_type"]=="goleag"].drop(columns=["bird_type"]).reset_index(drop=True)
    whteag_ds = ds[ds["bird_type"]=="whteag"].drop(columns=["bird_type"]).reset_index(drop=True)
    return goleag_ds, whteag_ds
    

In [3]:
hops_df = get_feature_group_as_df()
print("Len hopsworks ds",hops_df.shape[0])
hops_df = true_false_to_01(hops_df)
hops_df = sort_by_date(hops_df)
hops_df, REGION_MAPPING = encode_reigon(hops_df)
hops_df = drop_unused_columns(hops_df, drop_date = False)
hops_df = birdcount_binarization(hops_df)
print(hops_df.columns)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.76s) 
Len hopsworks ds 273350
Index(['region', 'observation_date', 'wind', 'rain', 'temperature',
       'bird_type', 'year', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'month_12', 'sighted_lag_1', 'sighted_lag_2',
       'sighted_lag_3', 'sighted_lag_4', 'sighted_lag_5', 'bird_count_binary'],
      dtype='object')


In [4]:
weather_df=features()
weather_df = to_hopsworks_df(weather_df)
weather_df = sort_by_date(weather_df)
REGION_MAP = get_reigon_mapping()
weather_df, __ = encode_reigon(weather_df,premade_mapping =REGION_MAP)
weather_df = true_false_to_01(weather_df)
print(weather_df.shape)
weather_df = make_feature_compatible(weather_df, reference_df=hops_df)
print(weather_df.shape)

(175, 19)
(350, 25)


In [5]:
mergd = fill_sighted_lag(weather_df,format_observation_date(hops_df))
goleag_ds, whteag_ds = divide_ds(mergd)
print(mergd.shape, goleag_ds.shape, whteag_ds.shape)

(350, 25) (175, 24) (175, 24)


## Infering the model

In [6]:
mr = project.get_model_registry()

# 2. Retrieve model metadata from Hopsworks (e.g., version 1)
hops_goldag_meta = mr.get_model("Goleag_model", version=1)
hops_whteag_meta = mr.get_model("Whteag_model", version=1)
# 3. Download the model files to a local directory
g_path = hops_goldag_meta.download()
w_path = hops_whteag_meta.download()

# 4. Re-initialize your model classes (ensure in_features matches your training data)
goldag_model_loaded = BirdPercentModel(in_features=22, hidden_layers=[32, 16, 1])
whteag_model_loaded = BirdPercentModel(in_features=22, hidden_layers=[64, 32, 1])

Downloading: 0.000%|          | 0/8209 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/17425 elapsed<00:00 remaining<?

In [15]:
def get_predictions_for_date(model: nn.Module, df: pd.DataFrame, date: int = 0) -> np.ndarray:
    model.eval()
    with torch.no_grad():
        input_tensor = df_to_tensor(df, date)
        outputs = model(input_tensor).squeeze().numpy()
    return outputs



def df_to_tensor(df: pd.DataFrame, date: int = 0) -> torch.Tensor:
    # Get the specific date value
    unique_dates = df['observation_date'].unique()
    target_date = unique_dates[date]
    
    # Filter and sort by region to ensure consistent row order
    day_df = df[df['observation_date'] == target_date].sort_values('region')
    
    # Drop non-feature columns
    # We use errors='ignore' in case columns were already removed
    features = day_df.drop(columns=['observation_date', 'bird_count_binary', 'bird_type'], errors='ignore')
    
    return torch.tensor(features.values.astype(np.float32), dtype=torch.float32)

def instert_update_rolling(df: pd.DataFrame, predictions: np.ndarray, date: int = 0) -> pd.DataFrame:
    unique_dates = df['observation_date'].unique()
    binary_preds = (predictions >= 0.5).astype(int)
    
    # Get regions for the current date in the same order as df_to_tensor
    current_date = unique_dates[date]
    current_day_mask = df['observation_date'] == current_date
    regions = df[current_day_mask].sort_values('region')['region'].values
    
    # Map predictions to regions
    pred_map = dict(zip(regions, binary_preds))

    # Update current day bird_count_binary with the predictions
    df.loc[current_day_mask, 'bird_count_binary'] = df.loc[current_day_mask, 'region'].map(pred_map)
    
    # Propagate to future lags
    for i in range(1, 6):
        if date + i < len(unique_dates):
            future_date = unique_dates[date + i]
            # Update each region's lag column for the future date
            mask = df['observation_date'] == future_date
            df.loc[mask, f'sighted_lag_{i}'] = df.loc[mask, 'region'].map(pred_map)
            
    return df

def predict(bird_df: pd.DataFrame, model: nn.Module) -> pd.DataFrame:
    region_mapping = {0: 'Dalsland', 1: 'Ångermanland', 2: 'Medelpad', 3: 'Jämtland', 4: 'Härjedalen', 5: 'Bohuslän', 6: 'Hälsingland', 7: 'Dalarna', 8: 'Norrbotten', 9: 'Blekinge', 10: 'Lappland', 11: 'Värmland', 12: 'Västmanland', 13: 'Gästrikland', 14: 'Småland', 15: 'Östergötland', 16: 'Närke', 17: 'Västerbotten', 18: 'Gotland', 19: 'Västergötland', 20: 'Halland', 21: 'Uppland', 22: 'Öland', 23: 'Södermanland', 24: 'Skåne'}
    
    bird_df = bird_df.copy()
    bird_df = bird_df.sort_values('observation_date')
    unique_dates = bird_df['observation_date'].unique()
    
    results = []
    
    for date_idx in range(len(unique_dates)):
        outputs = get_predictions_for_date(model, bird_df, date_idx)
        
        current_date = unique_dates[date_idx]
        # Sorting by region ensures the n-th model output matches the n-th region ID
        current_day_df = bird_df[bird_df['observation_date'] == current_date].sort_values('region')
        region_ids = current_day_df['region'].values
        
        for r_id, prob in zip(region_ids, outputs):
            results.append({
                'observation_date': current_date,
                'region': region_mapping.get(r_id, r_id), # Map ID to Name
                'probability': float(prob)
            })
        
        bird_df = instert_update_rolling(bird_df, outputs, date_idx)
    
    return pd.DataFrame(results)


whteag_preds = predict(whteag_ds, whteag_model_loaded)
whteag_preds["bird_type"] = "whteag"
goleag_preds = predict(goleag_ds, goldag_model_loaded)
goleag_preds["bird_type"] = "goleag"
preds = pd.concat([whteag_preds, goleag_preds], ignore_index=True)
preds = to_hopsworks_df(preds)

In [16]:
preds

,observation_date,region,probability,bird_type
0,2025-12-21,Dalsland,0.472138,whteag
1,2025-12-21,Ångermanland,0.553263,whteag
2,2025-12-21,Medelpad,0.490655,whteag
3,2025-12-21,Jämtland,0.484350,whteag
4,2025-12-21,Härjedalen,0.514317,whteag
...,...,...,...,...
345,2025-12-27,Halland,0.215116,goleag
346,2025-12-27,Uppland,0.191584,goleag
347,2025-12-27,Öland,0.199252,goleag
348,2025-12-27,Södermanland,0.186363,goleag


In [17]:
# code for adding it to hopsworks feature store
fs = project.get_feature_store()
fg = fs.get_or_create_feature_group(
    name="bird_sighting_predictions",
    version=1,
    description="Predicted probabilities of bird sightings by region and date",
    primary_key=["observation_date", "region", "bird_type"])

fg.insert(preds)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1327256/fs/1315910/fg/1840748


Uploading Dataframe: 100.00% |██████████| Rows 350/350 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bird_sighting_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1327256/jobs/named/bird_sighting_predictions_1_offline_fg_materialization/executions


(Job('bird_sighting_predictions_1_offline_fg_materialization', 'SPARK'), None)